## Quantum Cryptography
#### Python Script

In [162]:
# imports
import numpy as np
import pandas as pd
from numpy import random
import scipy
import matplotlib.pyplot as plt
import math
import endecrypt # might need to download this module


# + base is 0 and 90 degrees in the half wave plate
# x base is 45 and -45 degrees in the half wave plate

# See experiment instructions for theory explanation and experiment steps

In [183]:
# Functions

########################################################################################################################
# First Section - Alice & Bob without Eve
def random_without_eve(length):
    # Generate random base for Alice, Bob, and random bits for Alice
    # Change length for number of bits sent by Alice to Bob and size of base chosen
    
    alice_bit = random.randint(2, size=length)
    
    # for random number generation, 0 is + base and 1 is x base, NOT TO CONFUSE WITH BITS 0 AND 1
    alice_base = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    bob_base = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    
    return [alice_bit, alice_base, bob_base]



def simulation_without_eve(alice_base, alice_bit, bob_base):
    # simulation for bob's bits to compare to experiment
    # input is the generated random bases (Alice and Bob) and random bits (Alice)
    # The simulation is used as the 'theoretical' result to compare to the experimental results
    
    bob_bit = []
    n = len(alice_bit)
    
    for i in range(n):
        if alice_base[i] == bob_base[i]: 
            # if the bases are the matching for Alice and Bob, 100% that the bit is the same
            bob_bit.append(alice_bit[i])
        else: 
            # if the bases are not matching for Alice and Bob, 50% that the bit is the same,
            # therefore another random number is generated for this bit
            temp = random.randint(2, size=(1))
            bob_bit.append(temp[0])
    
    return bob_bit



def create_key(alice_base, alice_bits, bob_base, bob_bits):
    # function that creates the key for Alice and Bob
    
    n = len(alice_base)
    
    # initialize base array to compare between Alice and Bob
    matched_base = [None for j in range(n)]
    
    # initialize bit array to compare between alice and bob
    matched_bits = [None for j in range(n)]
    
    # counter to check how many bits are matching/not matching
    # The counter here could be used in order to check for errors during the experiment - if neccesary
    count_true = 0
    count_false = 0
    
    
    # check for matching bases
    # None values in the array are non matching bases
    for i in range(n):
        if alice_base[i] == bob_base[i]:
            matched_base[i] = alice_base[i]
    
    # check for matching bits
    # None values in the array are non matching bits
    for i in range(n):
        if matched_base[i] != None:
            if alice_bits[i] == bob_bits[i]:
                matched_bits[i] = True
                count_true += 1
            elif alice_bits[i] != bob_bits[i]:
                matched_bits[i] = False
                count_false += 1
    
    # count total of bits compared
    total_count = count_true + count_false
    
    accuracy = count_true / n # should be around 50%
    
    # create key
    # should be similar if alice_bits array is used, if there are no errors - we assume for the moment there are none
    res_key = [bob_bits[i] for i in range(n) if matched_base[i] != None]
    
    return [res_key, len(res_key), total_count, count_true, count_false, accuracy]



def shorten_key(data,key):
    # shortens key if it is longer than the given data
    # the key should be the same length as the data for the binary addition
    if len(key) < len(data):
        print('Error - key should be longer than the data')
        return 'Error!'
    return [key[i] for i in range(len(data))]



def encryption(data,key):
    # function which creates the encrypted message using the key created between Alice and Bob
    # using simple binary addition
    # Used by Alice
    
    if len(data) != len(key):
        # if the key is longer than the data, shorten the key
        key = shorten_key(data,key)
    
    encrypted_message = [None for j in range(len(data))]
    
    # binary addition
    for i in range(len(data)):
        if (data[i] == 0 and key[i] == 0) or (data[i] == 1 and key[i] == 1):
            encrypted_message[i] = 0
        elif (data[i] == 1 and key[i] == 0) or (data[i] == 0 and key[i] == 1):
            encrypted_message[i] = 1
            
    return encrypted_message



def decryption(message,key):
    # function which recreates the original message using the key created between Alice and Bob
    # The message argument here is the encrypted message Bob recieves from Alice
    # This function is the same as the encryption function, using simple binary addition
    # Used by Bob
    return encryption(message,key)


def string_to_binary(string):
    # finds the binary value of the message
    # Notice for this experiment purposes, the first two integers are discarded for the used message
    # Should generally be 3, but missing the first zero in this case
    # These first integers are used to differ from lower/upper case letters, which is not used in this experiment for the given key
    return endecrypt.encode(string, 'binary')
    
def binary_to_string(binary):
    # finds the string of the binary value
    # input should be a single string of 1's and 0's accordingly with whitespace between each letter
    # Should enter each letter with the first three integers: Lowercase - 011, Uppercase - 010
    return endecrypt.decode(binary, 'binary')

def string_converter(binary_lst):
    # this function creates an alphabet message from a the binary code
    word_length = 5 # as explained, every word is given as a 5 letter binary code for the message sent
    
    # split the list to different words
    temp = [binary_lst[i:i + 5] for i in range(0, len(binary_lst), 5)]

    bin_data = ''

    for i in range(len(temp)):
        temp[i] = [0,1,0] + temp[i] # add the uppercase digits, can change to lowercase using [0,1,1]
        str_temp = ' ' # notice the whitespace at the beginning of every letter
        res_temp = ' ' # another temporary variable
        for j in range(len(temp[i])): 
            # loop through every word
            str_temp = str(temp[i][j]) # change to string values
            res_temp += str_temp
        bin_data += res_temp
    
    bin_data = bin_data[1:] # delete the first whitespace from the string
    
    binary_values = bin_data.split() # split on whitespace to convert each letter separatley

    res_string = ""
    
    for binary_value in binary_values:
        temp_int = int(binary_value, 2) # create binary value of item
        temp_char = chr(temp_int) # find the letter using the binary alphabet
        res_string += temp_char
    
    return res_string

def binary_converter(string):
    # converts string or message to binary list which corresponds to the message Alice wants to send
    # initialize lists 
    temp_lst = []
    res_lst = []
    
    for character in string:
        # convert to binary
        temp_lst.append(bin(ord(character))[2:].zfill(8))
        
    for i in range(len(temp_lst)):
        # delete three first binary numbers as explained for message
        temp_lst[i] = temp_lst[i][3:]
        
    for j in range(len(temp_lst)):
        # create message as one list of binary numbers to send to Bob via the channel
        for k in range(len(temp_lst[j])):
            res_lst.append(int(temp_lst[j][k]))
    return res_lst



def compare_keys(key1,key2):
    # compare between keys for Alice and Bob
    if key1 != key2:
        return False
    return True


###########################################################################################################################
# Second Section - Alice & Bob with Eve

def random_with_eve(length):
    # Generate random base for Alice, Bob, Eve and random bits for Alice
    # Change length for number of bits sent by Alice to Bob and size of base chosen
    # Note that there are two bases for Eve which are generated randomly - incident base and transmitted base
    
    alice_bit = random.randint(2, size=length)
    
    # for random number generation, 0 is + base and 1 is x base, NOT TO CONFUSE WITH BITS 0 AND 1
    alice_base = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    bob_base = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    eve_base_inc = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    eve_base_tran = ['+' if i == 0 else 'x' for i in random.randint(2, size=(length))]
    
    return [alice_bit, alice_base, bob_base, eve_base_inc, eve_base_tran]

def simulation_with_eve(alice_base,alice_bit,bob_base,eve_base_inc,eve_base_tran):
    # simulation for bob's and eve's bits to compare to experiment
    # input is the generated random bases (Alice, Bob, and two Eve bases) and random bits (Alice)
    # The simulation is used as the 'theoretical' result to compare to the experimental results
    
    # Note here, used two different bases for Eve similar to the experiment itself to create real randomness,
    # where Eve does not really know the true base that Alice sends the bit in.
    
    bob_bit = []
    eve_bit = []
    
    n = len(alice_base)
    m = len(eve_base_tran)
    
    # Eve's part
    for i in range(n):
        if alice_base[i] == eve_base_inc[i]:
            eve_bit.append(alice_bit[i])
        else:
            temp = random.randint(2, size=(1))
            eve_bit.append(temp[0])
    
    # Bob's part
    # Eve's transmitted base
    for j in range(m):
        if eve_base_tran[j] == bob_base[j]:
            bob_bit.append(eve_bit[j])
        else:
            temp = random.randint(2, size=(1))
            bob_bit.append(temp[0])
    
    return [bob_bit, eve_bit]


# function which checks if Eve is listening on the channel between Alice and Bob
# theoretically, the accuracy should be around 25% if there is an evesdropper
# Note only Alice and Bob check their bases and bits
def check_eve(alice_base, bob_base, alice_bits, bob_bits):
    
    n = len(alice_base)
    
    # initialize base and bits to compare between alice and bob
    matched_base = [None for j in range(n)]
    matched_bits = [None for j in range(n)] 
    
    # counter to check how many bits are matching/not matching
    count_true = 0
    count_false = 0
    
    # Check for matching bases
    # None values are non matching bases
    for i in range(n):
        if alice_base[i] == bob_base[i]:
            matched_base[i] = alice_base[i]
    
    # Check for matching bits in the matching bases
    # None values are none matching bases for these bits
    for i in range(n):
        if matched_base[i] != None:
            if alice_bits[i] == bob_bits[i]:
                matched_bits[i] = True
                count_true += 1
            elif alice_bits[i] != bob_bits[i]:
                matched_bits[i] = False
                count_false += 1
    
    # count total of bits compared
    total_count = count_true + count_false
    
    accuracy = count_true / len(alice_bits) # should be around 25% for matching bases
    
    if accuracy >= 0.15 and accuracy <= 0.40: # should be approx 25% so range is chosen for some margin error
        print('Accuracy:',accuracy)
        print('Total Count of Bits Sent by Alice:',len(alice_bits))
        print('Total Count of matching base:',total_count)
        print('Number of bits Not matching:',count_false)
        print('Number of bits matching:',count_true)
        print('Eve is Evesdropping, Create New Key!')
        return
    else:
        print('Accuracy:',accuracy)
        print('Total Count of Bits Sent by Alice:',len(alice_bits))
        print('Total Count of matching base:',total_count)
        print('Number of bits Not matching:',count_false)
        print('Number of bits matching:',count_true)
        print('Key is safe, continue to encryption')
        return 
    

In [164]:
# Part A - Alice & Bob 18 bit example

# Simulation
alice_base_theory = ['x','x','+','x','+','+','+','x','x','+','x','x','x','+','+','x','+','x']
alice_bit_theory = [1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1]
bob_base_theory = ['+','x','x','x','+','+','+','x','+','x','x','+','+','x','+','+','+','x']

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)

bob_bit_theory = simulation_without_eve(alice_base_theory,alice_bit_theory,bob_base_theory)
print('\n','Bob bit theory:',bob_bit_theory)

# Experiment
alice_base_exp = ['x','x','+','x','+','+','+','x','x','+','x','x','x','+','+','x','+','x']
alice_bit_exp = [1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,1,0,1]
bob_base_exp = ['+','x','x','x','+','+','+','x','+','x','x','+','+','x','+','+','+','x']

# data from the experiment
bob_bit_exp = [1,0,1,1,1,0,0,1,1,0,0,1,0,1,0,1,0,1]
print('\n','Bob bit experiment:',bob_bit_exp)


# create keys
key_theory = create_key(alice_base_theory,alice_bit_theory,bob_base_theory,bob_bit_theory)
key_exp = create_key(alice_base_exp,alice_bit_exp,bob_base_exp,bob_bit_exp)

print('\n','Theory','\n','Key Theory:',key_theory[0],'\n','Number of matching base:',key_theory[2],'\n','Number of matching bits:',key_theory[3],'\n','Accuracy:',key_theory[5])
print('\n','Experiment','\n','Key Experiment:',key_exp[0],'\n','Number of matching base:',key_exp[2],'\n','Number of matching bits:',key_exp[3],'\n','Accuracy:',key_exp[5])

# print True if keys are similar, False if different
print('\n','Keys are the Same?:',compare_keys(key_theory[0],key_exp[0]),'\n')


# Send an encrypted message with key from experiment
# Our message for this example is EM

string_to_binary('EM')
binary_to_string('01000101 01001101')

# write message given the binary representation - without leading integers as explained above
message = binary_converter('EM')
print('Alice message:',message)

# Alice creates the encrypted message
encrypted_message = encryption(message,key_exp[0])
print('\n','Alice sends encrypted message:',encrypted_message)

# Bob's received bits (should be the same as Alice's encrypted message with base +) - from the experiment
bob_recieved = [0,1,0,0,1,1,1,1,0,0]
print('\n','Bob received:',bob_recieved)

# Bob recreates the message
decrypted_message = decryption(bob_recieved,key_exp[0])
print('\n','Bob recreates the message:',decrypted_message)

# after recreating the message
message_bob = string_converter(decrypted_message)
print('\n',"Bob's recreated string message:",message_bob)


 Alice Base: ['x', 'x', '+', 'x', '+', '+', '+', 'x', 'x', '+', 'x', 'x', 'x', '+', '+', 'x', '+', 'x']

 Alice Bit: [1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]

 Bob Base: ['+', 'x', 'x', 'x', '+', '+', '+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+', '+', 'x']

 Bob bit theory: [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1]

 Bob bit experiment: [1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]

 Theory 
 Key Theory: [0, 1, 1, 0, 0, 1, 0, 0, 0, 1] 
 Number of matching base: 10 
 Number of matching bits: 10 
 Accuracy: 0.5555555555555556

 Experiment 
 Key Experiment: [0, 1, 1, 0, 0, 1, 0, 0, 0, 1] 
 Number of matching base: 10 
 Number of matching bits: 10 
 Accuracy: 0.5555555555555556

 Keys are the Same?: True 

1000101 1001101
EM
Alice message: [0, 0, 1, 0, 1, 0, 1, 1, 0, 1]

 Alice sends encrypted message: [0, 1, 0, 0, 1, 1, 1, 1, 0, 0]

 Bob received: [0, 1, 0, 0, 1, 1, 1, 1, 0, 0]

 Bob recreates the message: [0, 0, 1, 0, 1, 0, 1, 1, 0, 1]

 Bob's rec

In [165]:
# Part B - Alice & Bob 52 bit example

# generate random numbers using the function
random_gen = random_without_eve(52)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]

# Simulation
# The base (Alice and Bob) and bits (Alice) given here are the ones that were generated randomly for this part
alice_base_theory = ['x','x','x','x','+','x','+','+','+','x','+','+','+','+','x','+','x','x','x','x','+','x','x','x','+','x','+','x','+','x','+','x','x','x','+','+','+','x','x','+','x','+','+','+','+','+','+','+','+','+','+','+']
alice_bit_theory = [1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,1,1,0,0,1,1,1,0,0,0,0,1,1]
bob_base_theory = ['x','x','x','+','+','x','+','x','+','x','x','+','+','x','+','+','x','+','x','x','+','x','x','+','+','x','x','x','x','+','+','x','x','+','+','+','+','+','+','x','+','x','+','+','+','+','x','+','x','+','x','+']

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)

bob_bit_theory = simulation_without_eve(alice_base_theory,alice_bit_theory,bob_base_theory)
print('\n','Bob bit theory:',bob_bit_theory)

# Experiment
alice_base_exp = ['x','x','x','x','+','x','+','+','+','x','+','+','+','+','x','+','x','x','x','x','+','x','x','x','+','x','+','x','+','x','+','x','x','x','+','+','+','x','x','+','x','+','+','+','+','+','+','+','+','+','+','+']
alice_bit_exp = [1,0,0,0,1,0,0,0,0,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,0,1,0,1,1,0,0,1,1,1,0,0,0,0,1,1]
bob_base_exp = ['x','x','x','+','+','x','+','x','+','x','x','+','+','x','+','+','x','+','x','x','+','x','x','+','+','x','x','x','x','+','+','x','x','+','+','+','+','+','+','x','+','x','+','+','+','+','x','+','x','+','x','+']

bob_bit_exp = [1,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,1,0,0,1,1,0,1,1,1,1,0,1,0,0,1]
print('\n','Bob bit experiment:',bob_bit_exp)

# create keys
key_theory = create_key(alice_base_theory,alice_bit_theory,bob_base_theory,bob_bit_theory)
key_exp = create_key(alice_base_exp,alice_bit_exp,bob_base_exp,bob_bit_exp)

print('\n','Theory','\n','Key Theory:',key_theory[0],'\n','Number of matching base:',key_theory[2],'\n','Number of matching bits:',key_theory[3],'\n','Accuracy:',key_theory[5])
print('\n','Experiment','\n','Key Experiment:',key_exp[0],'\n','Number of matching base:',key_exp[2],'\n','Number of matching bits:',key_exp[3],'\n','Accuracy:',key_exp[5])

# print True if keys are similar, False if different
print('\n','Keys are the Same?:',compare_keys(key_theory[0],key_exp[0]),'\n')


# Send an encrypted message with key from experiment
# Our message for this example is BOHR

string_to_binary('BOHR')
binary_to_string('01000010 01001111 01001000 01010010')

# write message given the binary representation - without leading integers as explained above
message = binary_converter('BOHR')
print('Alice message:',message)

# Alice creates the encrypted message
encrypted_message = encryption(message,key_exp[0])
print('\n','Alice sends encrypted message:',encrypted_message)

# Bob's received bits (should be the same as Alice's encrypted message with base +) - from the experiment
bob_recieved = [1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0]
print('\n','Bob received:',bob_recieved)

# Bob recreates the message
decrypted_message = decryption(bob_recieved,key_exp[0])
print('\n','Bob recreates the message:',decrypted_message)

# after recreating the message
message_bob_2 = string_converter(decrypted_message)
print('\n',"Bob's recreated string message:",message_bob_2)


 Alice Base: ['x', 'x', 'x', 'x', '+', 'x', '+', '+', '+', 'x', '+', '+', '+', '+', 'x', '+', 'x', 'x', 'x', 'x', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', '+', '+', 'x', 'x', '+', 'x', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+']

 Alice Bit: [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

 Bob Base: ['x', 'x', 'x', '+', '+', 'x', '+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+', 'x', '+', 'x', 'x', '+', 'x', 'x', '+', '+', 'x', 'x', 'x', 'x', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', 'x', '+', 'x', '+', '+', '+', '+', 'x', '+', 'x', '+', 'x', '+']

 Bob bit theory: [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]

 Bob bit experiment: [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

In [168]:
# Part C - Eve 18 bit example

# generate random numbers using the function
random_gen = random_with_eve(18)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)
print('\n','Eve Base Incident:',eve_base_inc_theory)
print('\n','Eve Base Transmitted:',eve_base_tran_theory)

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

print('\n','Bob bit theory:',bob_bit_theory)
print('\n','Eve bit theory:',eve_bit_theory)


 Alice Base: ['+', '+', '+', '+', 'x', '+', 'x', 'x', '+', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', 'x']

 Alice Bit: [0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0]

 Bob Base: ['+', 'x', 'x', 'x', 'x', 'x', '+', 'x', 'x', 'x', '+', 'x', '+', '+', 'x', 'x', 'x', 'x']

 Eve Base Incident: ['x', 'x', 'x', 'x', '+', 'x', '+', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', 'x', '+']

 Eve Base Transmitted: ['x', '+', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '+', '+', 'x', 'x', 'x', '+']

 Bob bit theory: [1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1]

 Eve bit theory: [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0]


In [169]:
# Part C - 18 bit example - Continue

# Experiment - Same random base and bit as simulation (For comparison)
alice_bit_exp = alice_bit_theory
alice_base_exp = alice_base_theory
bob_base_exp = bob_base_theory
eve_base_inc_exp = eve_base_inc_theory
eve_base_tran_exp = eve_base_tran_theory

# From experiment results
bob_bit_exp = [1,0,1,1,1,1,0,1,0,1,1,1,1,0,1,0,1,1]
eve_bit_exp = [1,1,1,1,1,1,1,1,0,1,0,1,1,0,1,0,1,1]
print('\n','Eve bit experiment:',eve_bit_exp) # Unnecassary for the experiment, only for explanatory reasons
print('\n','Bob bit experiment:',bob_bit_exp)


# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)
print('\n','Check For Eve from Experiment:','\n')
check_for_eve_exp = check_eve(alice_base_exp,bob_base_exp,alice_bit_exp,bob_bit_exp)


 Eve bit experiment: [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1]

 Bob bit experiment: [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1]

 Check For Eve from Simulation: 

Accuracy: 0.5
Total Count of matching base: 10
Number of bits Not matching: 5
Number of bits matching: 5
Eve is Evesdropping, Create New Key!

 Check For Eve from Experiment: 

Accuracy: 0.5
Total Count of matching base: 10
Number of bits Not matching: 5
Number of bits matching: 5
Eve is Evesdropping, Create New Key!


In [172]:
# Part D - Eve 52 bit example

# generate random numbers using the function
random_gen = random_with_eve(52)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)
print('\n','Eve Base Incident:',eve_base_inc_theory)
print('\n','Eve Base Transmitted:',eve_base_tran_theory)

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

print('\n','Bob bit theory:',bob_bit_theory)
print('\n','Eve bit theory:',eve_bit_theory)


 Alice Base: ['x', 'x', 'x', '+', 'x', 'x', 'x', '+', 'x', 'x', '+', '+', 'x', 'x', '+', 'x', '+', 'x', '+', '+', '+', 'x', '+', 'x', '+', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', '+', '+', 'x', 'x', 'x', '+', 'x', '+', '+', 'x', '+', 'x', 'x', 'x', '+', '+', 'x']

 Alice Bit: [0 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0
 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0]

 Bob Base: ['x', '+', '+', '+', 'x', '+', '+', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', '+', '+', 'x', 'x', 'x', '+', '+', 'x', 'x', '+', '+', 'x', 'x', '+', '+', '+', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', 'x', '+', 'x', '+', '+', '+', '+', '+', '+']

 Eve Base Incident: ['+', '+', '+', '+', 'x', 'x', '+', '+', 'x', '+', '+', '+', '+', 'x', 'x', '+', 'x', '+', 'x', 'x', '+', '+', 'x', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', 'x']

 Eve Base Transmitted: ['x', 'x', '+', '+', 'x', 

In [173]:
# Part D - 52 bit example - Continue

# Experiment - Same random base and bit as simulation (For comparison)
alice_bit_exp = alice_bit_theory
alice_base_exp = alice_base_theory
bob_base_exp = bob_base_theory
eve_base_inc_exp = eve_base_inc_theory
eve_base_tran_exp = eve_base_tran_theory

# From experiment results
bob_bit_exp = [1,0,1,0,1,0,1,1,0,0,0,1,1,0,1,0,1,0,1,0,1,1,1,0,1,0,0,1,1,1,0,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,1,0,0]
eve_bit_exp = [1,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0]
print('\n','Eve bit experiment:',eve_bit_exp) # Unnecassary for the experiment, only for explanatory reasons
print('\n','Bob bit experiment:',bob_bit_exp)


# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)
print('\n','Check For Eve from Experiment:','\n')
check_for_eve_exp = check_eve(alice_base_exp,bob_base_exp,alice_bit_exp,bob_bit_exp)


 Eve bit experiment: [1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0]

 Bob bit experiment: [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]

 Check For Eve from Simulation: 

Accuracy: 0.4166666666666667
Total Count of matching base: 24
Number of bits Not matching: 10
Number of bits matching: 14
Eve is Evesdropping, Create New Key!

 Check For Eve from Experiment: 

Accuracy: 0.4583333333333333
Total Count of matching base: 24
Number of bits Not matching: 11
Number of bits matching: 13
Eve is Evesdropping, Create New Key!


In [170]:
# Part E - Eve 30 bit example

# generate random numbers using the function
random_gen = random_with_eve(30)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)
print('\n','Eve Base Incident:',eve_base_inc_theory)
print('\n','Eve Base Transmitted:',eve_base_tran_theory)

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

print('\n','Bob bit theory:',bob_bit_theory)
print('\n','Eve bit theory:',eve_bit_theory)


 Alice Base: ['x', '+', '+', 'x', 'x', '+', '+', '+', 'x', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', '+', '+', '+', '+', '+', '+', '+', '+', 'x', '+', '+', 'x']

 Alice Bit: [1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0]

 Bob Base: ['+', 'x', '+', '+', '+', '+', '+', 'x', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', 'x', '+', '+', 'x', 'x', 'x', 'x', 'x', '+']

 Eve Base Incident: ['x', '+', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', '+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+']

 Eve Base Transmitted: ['+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+', '+', '+', '+', '+', 'x', 'x', '+', '+', 'x', '+', 'x', '+', '+', '+', 'x', '+', '+', 'x', 'x', 'x']

 Bob bit theory: [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1]

 Eve bit theory: [1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1]


In [171]:
# Part E - 30 bit example - Continue

# Experiment - Same random base and bit as simulation (For comparison)
alice_bit_exp = alice_bit_theory
alice_base_exp = alice_base_theory
bob_base_exp = bob_base_theory
eve_base_inc_exp = eve_base_inc_theory
eve_base_tran_exp = eve_base_tran_theory

# From experiment results
bob_bit_exp = [1,0,1,0,0,1,0,1,1,1,1,1,1,0,0,0,1,1,0,1,1,1,0,1,1,0,1,1,1,0]
eve_bit_exp = [1,0,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1]
print('\n','Eve bit experiment:',eve_bit_exp) # unnecassary for the experiment, only for explanatory reasons
print('\n','Bob bit experiment:',bob_bit_exp)


# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)
print('\n','Check For Eve from Experiment:','\n')
check_for_eve_exp = check_eve(alice_base_exp,bob_base_exp,alice_bit_exp,bob_bit_exp)


 Eve bit experiment: [1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]

 Bob bit experiment: [1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0]

 Check For Eve from Simulation: 

Accuracy: 0.3076923076923077
Total Count of matching base: 13
Number of bits Not matching: 4
Number of bits matching: 9
Eve is Evesdropping, Create New Key!

 Check For Eve from Experiment: 

Accuracy: 0.23076923076923078
Total Count of matching base: 13
Number of bits Not matching: 3
Number of bits matching: 10
Eve is Evesdropping, Create New Key!


In [166]:
# Part F - Eve 10 bit example

# generate random numbers using the function
random_gen = random_with_eve(10)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

print('\n','Alice Base:',alice_base_theory)
print('\n','Alice Bit:',alice_bit_theory)
print('\n','Bob Base:',bob_base_theory)
print('\n','Eve Base Incident:',eve_base_inc_theory)
print('\n','Eve Base Transmitted:',eve_base_tran_theory)

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

print('\n','Bob bit theory:',bob_bit_theory)
print('\n','Eve bit theory:',eve_bit_theory)


 Alice Base: ['+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', 'x']

 Alice Bit: [0 1 1 0 0 0 1 1 0 0]

 Bob Base: ['x', 'x', 'x', '+', 'x', 'x', '+', '+', '+', '+']

 Eve Base Incident: ['x', 'x', '+', 'x', '+', '+', '+', '+', 'x', '+']

 Eve Base Transmitted: ['+', '+', 'x', '+', '+', '+', '+', '+', 'x', 'x']

 Bob bit theory: [1, 1, 1, 0, 1, 0, 0, 0, 0, 1]

 Eve bit theory: [1, 1, 1, 0, 0, 0, 0, 0, 1, 1]


In [167]:
# Part F - 10 bit example - Continue

# Experiment - Same random base and bit as simulation (For comparison)
alice_bit_exp = alice_bit_theory
alice_base_exp = alice_base_theory
bob_base_exp = bob_base_theory
eve_base_inc_exp = eve_base_inc_theory
eve_base_tran_exp = eve_base_tran_theory

# From experiment results
bob_bit_exp = [1,1,1,0,0,0,1,1,0,1]
eve_bit_exp = [1,1,1,0,0,1,1,1,1,1]
print('\n','Eve bit experiment:',eve_bit_exp) # Unnecassary for the experiment, only for explanatory reasons
print('\n','Bob bit experiment:',bob_bit_exp)


# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)
print('\n','Check For Eve from Experiment:','\n')
check_for_eve_exp = check_eve(alice_base_exp,bob_base_exp,alice_bit_exp,bob_bit_exp)


 Eve bit experiment: [1, 1, 1, 0, 0, 1, 1, 1, 1, 1]

 Bob bit experiment: [1, 1, 1, 0, 0, 0, 1, 1, 0, 1]

 Check For Eve from Simulation: 

Accuracy: 0.0
Total Count of matching base: 3
Number of bits Not matching: 0
Number of bits matching: 3
Key is safe, continue to encryption

 Check For Eve from Experiment: 

Accuracy: 0.0
Total Count of matching base: 3
Number of bits Not matching: 0
Number of bits matching: 3
Key is safe, continue to encryption


In [184]:
# Part G (i) - 1000 bit example - SIMULATION ONLY

# generate random numbers using the function
random_gen = random_with_eve(1000)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)



 Check For Eve from Simulation: 

Accuracy: 0.317
Total Count of Bits Sent by Alice: 1000
Total Count of matching base: 514
Number of bits Not matching: 197
Number of bits matching: 317
Eve is Evesdropping, Create New Key!


In [185]:
# Part G (ii) - 10000 bit example - SIMULATION ONLY

# generate random numbers using the function
random_gen = random_with_eve(10000)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)


 Check For Eve from Simulation: 

Accuracy: 0.3077
Total Count of Bits Sent by Alice: 10000
Total Count of matching base: 4979
Number of bits Not matching: 1902
Number of bits matching: 3077
Eve is Evesdropping, Create New Key!


In [186]:
# Part G (iii) - 1000000 bit example - SIMULATION ONLY

# generate random numbers using the function
random_gen = random_with_eve(1000000)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)


 Check For Eve from Simulation: 

Accuracy: 0.312162
Total Count of Bits Sent by Alice: 1000000
Total Count of matching base: 499946
Number of bits Not matching: 187784
Number of bits matching: 312162
Eve is Evesdropping, Create New Key!


In [188]:
# Part G (iiii) - 10000000 bit example - SIMULATION ONLY

# generate random numbers using the function
random_gen = random_with_eve(10000000)

alice_bit_theory = random_gen[0]
alice_base_theory = random_gen[1]
bob_base_theory = random_gen[2]
eve_base_inc_theory = random_gen[3]
eve_base_tran_theory = random_gen[4]

# Simulation
sim_bit = simulation_with_eve(alice_base_theory,alice_bit_theory,bob_base_theory,eve_base_inc_theory,eve_base_tran_theory)
bob_bit_theory = sim_bit[0]
eve_bit_theory = sim_bit[1]

# Check for Eve
print('\n','Check For Eve from Simulation:','\n')
check_for_eve_theory = check_eve(alice_base_theory,bob_base_theory,alice_bit_theory,bob_bit_theory)


 Check For Eve from Simulation: 

Accuracy: 0.3127547
Total Count of Bits Sent by Alice: 10000000
Total Count of matching base: 5001162
Number of bits Not matching: 1873615
Number of bits matching: 3127547
Eve is Evesdropping, Create New Key!


In [182]:
# CODE WHICH CHECKS EFFICIENCY AND ACCURACY WHEN USING TWO BASES / ONE BASE FOR EVE

# trials
def simulation_with_eve_two_base(alice_base,alice_bit,bob_base,eve_base_inc,eve_base_tran):
    # simulation for bob's and eve's bits to compare to experiment
    # input is the generated random bases (Alice, Bob, and two Eve bases) and random bits (Alice)
    # The simulation is used as the 'theoretical' result to compare to the experimental results
    
    # Note here, used two different bases for Eve similar to the experiment itself to create real randomness,
    # where Eve does not really know the true base that Alice sends the bit in.
    
    bob_bit = []
    eve_bit = []
    
    n = len(alice_base)
    m = len(eve_base_tran)
    
    # Eve's part
    for i in range(n):
        if alice_base[i] == eve_base_inc[i]:
            eve_bit.append(alice_bit[i])
        else:
            temp = random.randint(2, size=(1))
            eve_bit.append(temp[0])
    
    # Bob's part
    # Eve's transmitted base
    for j in range(m):
        if eve_base_tran[j] == bob_base[j]:
            bob_bit.append(eve_bit[j])
        else:
            temp = random.randint(2, size=(1))
            bob_bit.append(temp[0])
    
    return [bob_bit, eve_bit]

def simulation_with_eve_one_base(alice_base,alice_bit,bob_base,eve_base):
    bob_bit = []
    eve_bit = []
    
    n = len(alice_base)
    m = len(eve_base)
    
    # Eve's part
    for i in range(n):
        if alice_base[i] == eve_base[i]:
            eve_bit.append(alice_bit[i])
        else:
            temp = random.randint(2, size=(1))
            eve_bit.append(temp[0])
    
    # Bob's part
    for j in range(m):
        if eve_base[j] == bob_base[j]:
            bob_bit.append(eve_bit[j])
        else:
            temp = random.randint(2, size=(1))
            bob_bit.append(temp[0])
    
    return [bob_bit, eve_bit]

# 10 bit
alice_base_10 = ['+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', 'x']
alice_bit_10 = [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
bob_base_10 = ['x', 'x', 'x', '+', 'x', 'x', '+', '+', '+', '+']
eve_base_inc_10 = ['x', 'x', '+', 'x', '+', '+', '+', '+', 'x', '+']
eve_base_tran_10 = ['+', '+', 'x', '+', '+', '+', '+', '+', 'x', 'x']

# 30 bit
alice_base_30 = ['x', '+', '+', 'x', 'x', '+', '+', '+', 'x', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', '+', '+', '+', '+', '+', '+', '+', '+', 'x', '+', '+', 'x']
alice_bit_30 = [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
bob_base_30 = ['+', 'x', '+', '+', '+', '+', '+', 'x', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', 'x', '+', '+', 'x', 'x', 'x', 'x', 'x', '+']
eve_base_inc_30 = ['x', '+', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', '+', '+', 'x', '+', 'x', '+', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+']
eve_base_tran_30 = ['+', 'x', '+', 'x', 'x', '+', '+', 'x', '+', '+', '+', '+', '+', '+', 'x', 'x', '+', '+', 'x', '+', 'x', '+', '+', '+', 'x', '+', '+', 'x', 'x', 'x']

# 52 bit
alice_base_52 = ['x', 'x', 'x', '+', 'x', 'x', 'x', '+', 'x', 'x', '+', '+', 'x', 'x', '+', 'x', '+', 'x', '+', '+', '+', 'x', '+', 'x', '+', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', '+', '+', '+', 'x', 'x', 'x', '+', 'x', '+', '+', 'x', '+', 'x', 'x', 'x', '+', '+', 'x']
alice_bit_52 = [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0]
bob_base_52 = ['x', '+', '+', '+', 'x', '+', '+', '+', '+', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', '+', '+', 'x', 'x', 'x', '+', '+', 'x', 'x', '+', '+', 'x', 'x', '+', '+', '+', '+', 'x', 'x', 'x', '+', 'x', '+', 'x', 'x', '+', 'x', '+', '+', '+', '+', '+', '+']
eve_base_inc_52 = ['+', '+', '+', '+', 'x', 'x', '+', '+', 'x', '+', '+', '+', '+', 'x', 'x', '+', 'x', '+', 'x', 'x', '+', '+', 'x', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x', 'x', 'x', 'x', '+', '+', '+', '+', '+', 'x', 'x', 'x', 'x', '+', '+', 'x', '+', 'x', 'x']
eve_base_tran_52 = ['x', 'x', '+', '+', 'x', 'x', '+', '+', 'x', '+', 'x', '+', '+', 'x', '+', 'x', '+', '+', 'x', 'x', 'x', 'x', 'x', '+', '+', 'x', '+', '+', '+', 'x', '+', 'x', '+', 'x', 'x', '+', '+', '+', '+', 'x', '+', '+', 'x', 'x', '+', '+', 'x', 'x', '+', '+', 'x', '+']

print('\n','10 bit')
print('\n','Two bases for Eve:')
sim_bit_10_two = simulation_with_eve_two_base(alice_base_10,alice_bit_10,bob_base_10,eve_base_inc_10,eve_base_tran_10)
bob_bit_10_two = sim_bit_10_two[0]
eve_bit_10_two = sim_bit_10_two[1]
print('\n','Bob bit theory:',bob_bit_10_two)
print('\n','Eve bit theory:',eve_bit_10_two)
check_eve(alice_base_10,bob_base_10,alice_bit_10,bob_bit_10_two)
print('\n','One base for Eve:')
sim_bit_10_one = simulation_with_eve_one_base(alice_base_10,alice_bit_10,bob_base_10,eve_base_inc_10)
bob_bit_10_one = sim_bit_10_one[0]
eve_bit_10_one = sim_bit_10_one[1]
print('\n','Bob bit theory:',bob_bit_10_one)
print('\n','Eve bit theory:',eve_bit_10_one)
check_eve(alice_base_10,bob_base_10,alice_bit_10,bob_bit_10_one)

print('\n','30 bit')
print('\n','Two bases for Eve:')
sim_bit_30_two = simulation_with_eve_two_base(alice_base_30,alice_bit_30,bob_base_30,eve_base_inc_30,eve_base_tran_30)
bob_bit_30_two = sim_bit_30_two[0]
eve_bit_30_two = sim_bit_30_two[1]
print('\n','Bob bit theory:',bob_bit_30_two)
print('\n','Eve bit theory:',eve_bit_30_two)
check_eve(alice_base_30,bob_base_30,alice_bit_30,bob_bit_30_two)
print('\n','One base for Eve:')
sim_bit_30_one = simulation_with_eve_one_base(alice_base_30,alice_bit_30,bob_base_30,eve_base_inc_30)
bob_bit_30_one = sim_bit_30_one[0]
eve_bit_30_one = sim_bit_30_one[1]
print('\n','Bob bit theory:',bob_bit_30_one)
print('\n','Eve bit theory:',eve_bit_30_one)
check_eve(alice_base_30,bob_base_30,alice_bit_30,bob_bit_30_one)

print('\n','52 bit')
print('\n','Two bases for Eve:')
sim_bit_52_two = simulation_with_eve_two_base(alice_base_52,alice_bit_52,bob_base_52,eve_base_inc_52,eve_base_tran_52)
bob_bit_52_two = sim_bit_52_two[0]
eve_bit_52_two = sim_bit_52_two[1]
print('\n','Bob bit theory:',bob_bit_52_two)
print('\n','Eve bit theory:',eve_bit_52_two)
check_eve(alice_base_52,bob_base_52,alice_bit_52,bob_bit_52_two)
print('\n','One base for Eve:')
sim_bit_52_one = simulation_with_eve_one_base(alice_base_52,alice_bit_52,bob_base_52,eve_base_inc_52)
bob_bit_52_one = sim_bit_52_one[0]
eve_bit_52_one = sim_bit_52_one[1]
print('\n','Bob bit theory:',bob_bit_52_one)
print('\n','Eve bit theory:',eve_bit_52_one)
check_eve(alice_base_52,bob_base_52,alice_bit_52,bob_bit_52_one)



 10 bit

 Two bases for Eve:

 Bob bit theory: [1, 0, 1, 0, 1, 1, 1, 1, 1, 1]

 Eve bit theory: [1, 1, 1, 0, 0, 0, 1, 1, 1, 0]
Accuracy: 0.3
Total Count of Bits Sent by Alice: 10
Total Count of matching base: 3
Number of bits Not matching: 3
Number of bits matching: 0
Eve is Evesdropping, Create New Key!

 One base for Eve:

 Bob bit theory: [1, 1, 0, 1, 0, 0, 1, 1, 1, 1]

 Eve bit theory: [1, 1, 1, 0, 0, 0, 1, 1, 1, 1]
Accuracy: 0.1
Total Count of Bits Sent by Alice: 10
Total Count of matching base: 3
Number of bits Not matching: 1
Number of bits matching: 2
Key is safe, continue to encryption

 30 bit

 Two bases for Eve:

 Bob bit theory: [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]

 Eve bit theory: [1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
Accuracy: 0.2
Total Count of Bits Sent by Alice: 30
Total Count of matching base: 13
Number of bits Not matching: 6
Number of bits matching: 7
Eve is